# Dataset Flexível com MapBiomas para Análise de Uso da Terra

## Extração com dados de satélite e agregação flexível

Este notebook extrai dados da [Base dos Dados](https://basedosdados.org/) usando MapBiomas (dados de satélite) para identificar áreas dedicadas a diferentes culturas, com opção de agregação por município, microrregião ou AMC.

### Principais melhorias:
- ✅ **MapBiomas**: Dados de satélite mais precisos que autodeclaração
- ✅ **Agregação flexível**: Município, Microrregião ou AMC
- ✅ **Cobertura completa**: Todas as unidades geográficas incluídas
- ✅ **Múltiplas culturas**: Áreas de agricultura, cana, soja, café, cítricos e outras


## 0. Configuração e Setup

Configure aqui os parâmetros principais da extração:


In [1]:
# Importar bibliotecas
import basedosdados as bd
import pandas as pd
import numpy as np
import os
from datetime import datetime

# CONFIGURAÇÕES PRINCIPAIS
PROJECT_ID = "bdsdasd"
ANOS = list(range(2003, 2022))

# CONFIGURAÇÃO DE AGREGAÇÃO
# Escolha uma das opções: 'municipio', 'microrregiao', 'amc'
NIVEL_AGREGACAO = 'microrregiao'  # ⚙️ ALTERE AQUI!

# CONFIGURAÇÃO DO PRODUTO (apenas para dados PAM, não MapBiomas)
# MapBiomas sempre exporta todas as categorias
PRODUTO_PAM = 'Cana-de-açúcar'  # Exemplos: 'Soja', 'Milho', 'Café (beneficiado)'

# Criar diretórios
os.makedirs("output", exist_ok=True)
os.makedirs("data", exist_ok=True)

# Validar configuração
assert NIVEL_AGREGACAO in ['municipio', 'microrregiao', 'amc'], "Nível de agregação inválido!"

print("✅ Configuração completa!")
print(f"📅 Período: {ANOS[0]}-{ANOS[-1]}")
print(f"🎯 Nível de agregação: {NIVEL_AGREGACAO.upper()}")
print(f"🌾 Produto PAM: {PRODUTO_PAM}")
print(f"🛰️ Categorias MapBiomas: Agricultura, Cana, Soja, Café, Cítricos, Outras Lavouras")


✅ Configuração completa!
📅 Período: 2003-2021
🎯 Nível de agregação: MICRORREGIAO
🌾 Produto PAM: Cana-de-açúcar
🛰️ Categorias MapBiomas: Agricultura, Cana, Soja, Café, Cítricos, Outras Lavouras


## 1. Mapeamentos Geográficos

Obter os mapeamentos necessários para agregação:


In [2]:
# PASSO 1.1: Obter diretório de municípios
query_municipios = """
SELECT 
    id_municipio,
    id_microrregiao,
    nome AS nome_municipio,
    sigla_uf
FROM 
    `basedosdados.br_bd_diretorios_brasil.municipio`
"""

print("🗺️ Baixando diretório de municípios...")
df_municipios = bd.read_sql(query_municipios, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_municipios):,} municípios mapeados")
print(f"✅ {df_municipios['id_microrregiao'].nunique()} microrregiões identificadas")

# PASSO 1.2: Obter mapeamento AMC se necessário
if NIVEL_AGREGACAO == 'amc':
    # Query AMC ajustada
    query_amc = f"""
    SELECT DISTINCT
        id_amc,
        id_municipio
    FROM 
        `basedosdados.br_ibge_amc.municipio_de_para`
    WHERE 
        ano_de <= {ANOS[0]} 
        AND ano_para >= {ANOS[-1]}
    """
    
    print("\n🗺️ Baixando mapeamento AMC...")
    df_amc = bd.read_sql(query_amc, billing_project_id=PROJECT_ID)
    
    # Adicionar AMC ao dataframe de municípios
    df_municipios = df_municipios.merge(df_amc, on='id_municipio', how='left')
    print(f"✅ {df_amc['id_amc'].nunique()} AMCs identificadas")


🗺️ Baixando diretório de municípios...
Downloading: 100%|██████████|
✅ 5,570 municípios mapeados
✅ 558 microrregiões identificadas


## 2. Dados MapBiomas (Uso da Terra)

Extrair dados de cobertura da terra por satélite:


In [4]:
# PASSO 2: Baixar dados MapBiomas
# IDs das categorias de interesse
CATEGORIAS_MAPBIOMAS = {
    '18': 'agricultura',
    '19': 'lavoura_temporaria',
    '39': 'soja',
    '20': 'cana',
    '41': 'outras_lavouras_temporarias',
    '46': 'cafe',
    '47': 'citricos'
}

ids_categorias = list(CATEGORIAS_MAPBIOMAS.keys())
ids_str = ', '.join([f"'{id}'" for id in ids_categorias])

query_mapbiomas = f"""
SELECT
    ano,
    id_municipio,
    id_classe,
    area
FROM 
    `basedosdados.br_mapbiomas_estatisticas.cobertura_municipio_classe`
WHERE 
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
    AND id_classe IN ({ids_str})
"""

print(f"🛰️ Baixando dados MapBiomas para {len(CATEGORIAS_MAPBIOMAS)} categorias...")
df_mapbiomas_mun = bd.read_sql(query_mapbiomas, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_mapbiomas_mun):,} registros de cobertura")

# Pivotar para ter uma coluna por categoria
df_mapbiomas_mun['categoria'] = df_mapbiomas_mun['id_classe'].map(CATEGORIAS_MAPBIOMAS)
df_mapbiomas_pivot = df_mapbiomas_mun.pivot_table(
    index=['ano', 'id_municipio'],
    columns='categoria',
    values='area',
    fill_value=0
).reset_index()

# Adicionar prefixo às colunas
cols_mapbiomas = [col for col in df_mapbiomas_pivot.columns if col not in ['ano', 'id_municipio']]
for col in cols_mapbiomas:
    df_mapbiomas_pivot[f'area_{col}'] = df_mapbiomas_pivot[col]
    df_mapbiomas_pivot.drop(col, axis=1, inplace=True)

print(f"✅ Dados pivotados: {len(df_mapbiomas_pivot):,} registros município-ano")


🛰️ Baixando dados MapBiomas para 7 categorias...
Downloading: 100%|██████████|
✅ 214,548 registros de cobertura
✅ Dados pivotados: 94,737 registros município-ano


## 3. Estações Meteorológicas

Identificar tratamento por estações automáticas:


In [5]:
# PASSO 3: Estações meteorológicas
query_estacoes = """
SELECT
    e.id_municipio,
    e.id_estacao,
    e.estacao AS nome_estacao,
    EXTRACT(YEAR FROM e.data_fundacao) AS ano_fundacao
FROM
    `basedosdados.br_inmet_bdmep.estacao` e
WHERE
    e.data_fundacao IS NOT NULL
    AND e.id_municipio IS NOT NULL
"""

print("🌡️ Baixando dados de estações...")
df_estacoes_mun = bd.read_sql(query_estacoes, billing_project_id=PROJECT_ID)

# Adicionar informação geográfica
df_estacoes_full = df_estacoes_mun.merge(
    df_municipios, 
    on='id_municipio', 
    how='left'
)

print(f"✅ {df_estacoes_full['id_estacao'].nunique()} estações identificadas")


🌡️ Baixando dados de estações...
Downloading: 100%|██████████|
✅ 610 estações identificadas


## 4. População e PIB

Dados socioeconômicos:


In [6]:
# PASSO 4.1: População
query_pop = f"""
SELECT
    ano,
    id_municipio,
    populacao
FROM
    `basedosdados.br_ibge_populacao.municipio`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
"""

print("👥 Baixando dados de população...")
df_pop_mun = bd.read_sql(query_pop, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_pop_mun):,} registros")

# PASSO 4.2: PIB
query_pib = f"""
SELECT
    ano,
    id_municipio,
    pib,
    va_agropecuaria
FROM
    `basedosdados.br_ibge_pib.municipio`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
"""

print("\n💰 Baixando dados de PIB...")
df_pib_mun = bd.read_sql(query_pib, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_pib_mun):,} registros")


👥 Baixando dados de população...
Downloading: 100%|██████████|
✅ 105,776 registros

💰 Baixando dados de PIB...
Downloading: 100%|██████████|
✅ 105,830 registros


## 5. Produção Agrícola (PAM)

Dados de produção para o produto selecionado:


In [7]:
# PASSO 5: Dados PAM para produto específico
query_pam = f"""
SELECT
    ano,
    id_municipio,
    area_plantada,
    area_colhida,
    quantidade_produzida,
    rendimento_medio_producao AS produtividade,
    valor_producao
FROM
    `basedosdados.br_ibge_pam.lavoura_temporaria`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
    AND produto = '{PRODUTO_PAM}'
"""

print(f"🌾 Baixando dados PAM para: {PRODUTO_PAM}...")
df_pam_mun = bd.read_sql(query_pam, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_pam_mun):,} registros de produção")

# Renomear colunas para evitar conflitos
df_pam_mun.columns = [
    'ano', 'id_municipio',
    'pam_area_plantada', 'pam_area_colhida', 
    'pam_quantidade_produzida', 'pam_produtividade',
    'pam_valor_producao'
]


🌾 Baixando dados PAM para: Cana-de-açúcar...
Downloading: 100%|██████████|
✅ 105,697 registros de produção


## 6. Agregação dos Dados

Agregar todos os dados conforme o nível escolhido:


In [8]:
# Definir função de agregação baseada no nível escolhido
def get_id_column():
    if NIVEL_AGREGACAO == 'municipio':
        return 'id_municipio'
    elif NIVEL_AGREGACAO == 'microrregiao':
        return 'id_microrregiao'
    else:  # amc
        return 'id_amc'

id_col = get_id_column()
print(f"🎯 Agregando por: {id_col}")

# Combinar todos os dados municipais primeiro
df_base = df_municipios[['id_municipio', 'id_microrregiao', 'sigla_uf']].copy()
if NIVEL_AGREGACAO == 'amc':
    df_base = df_base[['id_municipio', 'id_microrregiao', 'sigla_uf', 'id_amc']]

# Criar lista de todos os anos e unidades
from itertools import product

if NIVEL_AGREGACAO == 'municipio':
    all_units = df_base['id_municipio'].unique()
elif NIVEL_AGREGACAO == 'microrregiao':
    all_units = df_base['id_microrregiao'].unique()
else:  # amc
    all_units = df_base['id_amc'].dropna().unique()

# Criar painel completo
painel = pd.DataFrame(
    list(product(all_units, ANOS)), 
    columns=[id_col, 'ano']
)

print(f"✅ Painel criado: {len(painel):,} observações ({len(all_units)} unidades × {len(ANOS)} anos)")


🎯 Agregando por: id_microrregiao
✅ Painel criado: 10,602 observações (558 unidades × 19 anos)


In [9]:
# AGREGAR MAPBIOMAS
print("\n🛰️ Agregando dados MapBiomas...")
df_mapbiomas_mapped = df_mapbiomas_pivot.merge(df_base, on='id_municipio', how='left')

# Colunas de área para agregar
area_cols = [col for col in df_mapbiomas_mapped.columns if col.startswith('area_')]

if NIVEL_AGREGACAO == 'municipio':
    df_mapbiomas_agg = df_mapbiomas_mapped[['ano', 'id_municipio'] + area_cols]
else:
    # Agregar por microrregião ou AMC
    agg_dict = {col: 'sum' for col in area_cols}
    df_mapbiomas_agg = df_mapbiomas_mapped.groupby(['ano', id_col]).agg(agg_dict).reset_index()

painel = painel.merge(df_mapbiomas_agg, on=['ano', id_col], how='left')

# Preencher zeros onde não há dados
for col in area_cols:
    painel[col] = painel[col].fillna(0)

print(f"✅ MapBiomas agregado: {len(area_cols)} categorias de uso da terra")



🛰️ Agregando dados MapBiomas...
✅ MapBiomas agregado: 5 categorias de uso da terra


In [10]:
# AGREGAR ESTAÇÕES
print("\n🌡️ Agregando estações meteorológicas...")

# Agregar estações por unidade geográfica
if NIVEL_AGREGACAO == 'municipio':
    df_estacoes_agg = df_estacoes_full.groupby('id_municipio').agg({
        'ano_fundacao': 'min',
        'id_estacao': 'count'
    }).reset_index()
    df_estacoes_agg.columns = ['id_municipio', 'primeiro_ano_estacao', 'num_estacoes']
else:
    df_estacoes_agg = df_estacoes_full.groupby(id_col).agg({
        'ano_fundacao': 'min',
        'id_estacao': 'count'
    }).reset_index()
    df_estacoes_agg.columns = [id_col, 'primeiro_ano_estacao', 'num_estacoes']

# Adicionar ao painel
painel = painel.merge(df_estacoes_agg[[id_col, 'primeiro_ano_estacao']], on=id_col, how='left')

# Criar variáveis de tratamento
painel['primeiro_ano_tratamento'] = painel['primeiro_ano_estacao'].fillna(0).astype(int)
painel['tratado'] = (painel['primeiro_ano_tratamento'] != 0).astype(int)
painel['pos_tratamento'] = (
    (painel['ano'] >= painel['primeiro_ano_tratamento']) & 
    (painel['tratado'] == 1)
).astype(int)

print(f"✅ Unidades tratadas: {painel[painel['tratado'] == 1][id_col].nunique()}")
print(f"✅ Unidades controle: {painel[painel['tratado'] == 0][id_col].nunique()}")



🌡️ Agregando estações meteorológicas...
✅ Unidades tratadas: 394
✅ Unidades controle: 164


In [11]:
# AGREGAR POPULAÇÃO E PIB
print("\n💰 Agregando dados socioeconômicos...")

# Adicionar informação geográfica aos dados
df_pop_mapped = df_pop_mun.merge(df_base, on='id_municipio', how='left')
df_pib_mapped = df_pib_mun.merge(df_base, on='id_municipio', how='left')

if NIVEL_AGREGACAO == 'municipio':
    df_pop_agg = df_pop_mapped[['ano', 'id_municipio', 'populacao']]
    df_pop_agg.columns = ['ano', 'id_municipio', 'populacao_total']
    
    df_pib_agg = df_pib_mapped[['ano', 'id_municipio', 'pib', 'va_agropecuaria']]
    df_pib_agg.columns = ['ano', 'id_municipio', 'pib_total', 'pib_agropecuario']
else:
    # Agregar
    df_pop_agg = df_pop_mapped.groupby(['ano', id_col])['populacao'].sum().reset_index()
    df_pop_agg.columns = ['ano', id_col, 'populacao_total']
    
    df_pib_agg = df_pib_mapped.groupby(['ano', id_col]).agg({
        'pib': 'sum',
        'va_agropecuaria': 'sum'
    }).reset_index()
    df_pib_agg.columns = ['ano', id_col, 'pib_total', 'pib_agropecuario']

# Adicionar ao painel
painel = painel.merge(df_pop_agg, on=['ano', id_col], how='left')
painel = painel.merge(df_pib_agg, on=['ano', id_col], how='left')

# Calcular PIB per capita
painel['pib_per_capita'] = np.where(
    painel['populacao_total'] > 0,
    painel['pib_total'] / painel['populacao_total'],
    np.nan
)

print(f"✅ Dados socioeconômicos agregados")



💰 Agregando dados socioeconômicos...
✅ Dados socioeconômicos agregados


In [12]:
# AGREGAR PAM
print("\n🌾 Agregando dados PAM...")

df_pam_mapped = df_pam_mun.merge(df_base, on='id_municipio', how='left')

if NIVEL_AGREGACAO == 'municipio':
    df_pam_agg = df_pam_mapped[[
        'ano', 'id_municipio',
        'pam_area_plantada', 'pam_area_colhida', 
        'pam_quantidade_produzida', 'pam_produtividade',
        'pam_valor_producao'
    ]]
else:
    # Para produtividade, fazer média ponderada pela área
    df_pam_agg = df_pam_mapped.groupby(['ano', id_col]).agg({
        'pam_area_plantada': 'sum',
        'pam_area_colhida': 'sum',
        'pam_quantidade_produzida': 'sum',
        'pam_valor_producao': 'sum'
    }).reset_index()
    
    # Calcular produtividade média ponderada
    df_pam_agg['pam_produtividade'] = np.where(
        df_pam_agg['pam_area_colhida'] > 0,
        df_pam_agg['pam_quantidade_produzida'] / df_pam_agg['pam_area_colhida'],
        0
    )

# Adicionar ao painel
painel = painel.merge(df_pam_agg, on=['ano', id_col], how='left')

# Preencher zeros onde não há produção
pam_cols = [col for col in painel.columns if col.startswith('pam_')]
for col in pam_cols:
    painel[col] = painel[col].fillna(0)

print(f"✅ Dados PAM agregados para {PRODUTO_PAM}")



🌾 Agregando dados PAM...
✅ Dados PAM agregados para Cana-de-açúcar


In [13]:
# Adicionar UF ao painel
if NIVEL_AGREGACAO == 'municipio':
    uf_info = df_base[['id_municipio', 'sigla_uf']].drop_duplicates()
    painel = painel.merge(uf_info, on='id_municipio', how='left')
elif NIVEL_AGREGACAO == 'microrregiao':
    uf_info = df_base.groupby('id_microrregiao')['sigla_uf'].first().reset_index()
    painel = painel.merge(uf_info, on='id_microrregiao', how='left')
else:  # amc
    # Para AMC, pegar a UF mais comum
    uf_info = df_base.groupby('id_amc')['sigla_uf'].agg(lambda x: x.mode()[0] if len(x) > 0 else np.nan).reset_index()
    painel = painel.merge(uf_info, on='id_amc', how='left')

# Remover observações com dados faltantes críticos
print(f"\n🧹 Removendo observações com dados faltantes...")
print(f"   - Antes: {len(painel):,} observações")

# Remover onde não temos população ou PIB
painel_final = painel.dropna(subset=['populacao_total', 'pib_total'])

print(f"   - Após remover NAs (população/PIB): {len(painel_final):,} observações")
print(f"   - Removidas: {len(painel) - len(painel_final):,} observações ({(len(painel) - len(painel_final))/len(painel)*100:.1f}%)")



🧹 Removendo observações com dados faltantes...
   - Antes: 10,602 observações
   - Após remover NAs (população/PIB): 10,602 observações
   - Removidas: 0 observações (0.0%)


In [14]:
# Estatísticas do dataset final
print("\n📊 ESTATÍSTICAS DO DATASET FINAL")
print("=" * 50)
print(f"Nível de agregação: {NIVEL_AGREGACAO.upper()}")
print(f"Período: {ANOS[0]}-{ANOS[-1]}")
print(f"Produto PAM: {PRODUTO_PAM}")
print(f"\nDimensões: {len(painel_final):,} observações × {len(painel_final.columns)} variáveis")
print(f"Unidades geográficas: {painel_final[id_col].nunique():,}")
print(f"Estados: {painel_final['sigla_uf'].nunique()}")

print(f"\n🌡️ TRATAMENTO (Estações Meteorológicas):")
print(f"   - Unidades tratadas: {painel_final[painel_final['tratado'] == 1][id_col].nunique():,}")
print(f"   - Unidades controle: {painel_final[painel_final['tratado'] == 0][id_col].nunique():,}")
print(f"   - Observações pós-tratamento: {painel_final['pos_tratamento'].sum():,}")

print(f"\n🛰️ USO DA TERRA (MapBiomas - hectares):")
for col in area_cols:
    area_total = painel_final[col].sum()
    if area_total > 0:
        nome_categoria = col.replace('area_', '').replace('_', ' ').title()
        print(f"   - {nome_categoria}: {area_total:,.0f} ha total")

print(f"\n🌾 PRODUÇÃO PAM ({PRODUTO_PAM}):")
print(f"   - Unidades produtoras: {(painel_final['pam_area_plantada'] > 0).sum():,}")
print(f"   - Área plantada total: {painel_final['pam_area_plantada'].sum():,.0f} ha")
print(f"   - Produção total: {painel_final['pam_quantidade_produzida'].sum():,.0f} ton")
print(f"   - Produtividade média: {painel_final[painel_final['pam_produtividade'] > 0]['pam_produtividade'].mean():,.1f} kg/ha")

print(f"\n💰 SOCIOECONÔMICO:")
print(f"   - População total: {painel_final.groupby('ano')['populacao_total'].sum().iloc[-1]:,.0f} (último ano)")
print(f"   - PIB per capita médio: R$ {painel_final['pib_per_capita'].mean():,.2f}")
print(f"   - Participação agropecuária no PIB: {(painel_final['pib_agropecuario'].sum() / painel_final['pib_total'].sum() * 100):.1f}%")



📊 ESTATÍSTICAS DO DATASET FINAL
Nível de agregação: MICRORREGIAO
Período: 2003-2021
Produto PAM: Cana-de-açúcar

Dimensões: 10,602 observações × 21 variáveis
Unidades geográficas: 558
Estados: 27

🌡️ TRATAMENTO (Estações Meteorológicas):
   - Unidades tratadas: 394
   - Unidades controle: 164
   - Observações pós-tratamento: 5,582

🛰️ USO DA TERRA (MapBiomas - hectares):
   - Cafe: 192,696 ha total
   - Cana: 1,386,254 ha total
   - Citricos: 47,513 ha total
   - Outras Lavouras Temporarias: 1,670,675 ha total
   - Soja: 6,099,569 ha total

🌾 PRODUÇÃO PAM (Cana-de-açúcar):
   - Unidades produtoras: 9,134
   - Área plantada total: 167,419,126 ha
   - Produção total: 12,527,286,398 ton
   - Produtividade média: 51.5 kg/ha

💰 SOCIOECONÔMICO:
   - População total: 213,317,639 (último ano)
   - PIB per capita médio: R$ 17,857.00
   - Participação agropecuária no PIB: 4.8%


## 8. Exportar Dataset Final


In [15]:
# Organizar colunas na ordem desejada
cols_base = ['ano', id_col, 'sigla_uf', 'primeiro_ano_tratamento', 'tratado', 'pos_tratamento']
cols_pam = [col for col in painel_final.columns if col.startswith('pam_')]
cols_socio = ['populacao_total', 'pib_total', 'pib_per_capita', 'pib_agropecuario']
cols_mapbiomas = [col for col in painel_final.columns if col.startswith('area_')]

# Remover coluna temporária se existir
if 'primeiro_ano_estacao' in painel_final.columns:
    painel_final = painel_final.drop('primeiro_ano_estacao', axis=1)

# Ordenar colunas
cols_order = cols_base + cols_pam + cols_socio + cols_mapbiomas
cols_final = [col for col in cols_order if col in painel_final.columns]
df_export = painel_final[cols_final].sort_values([id_col, 'ano'])

# Nome do arquivo
timestamp = datetime.now().strftime('%Y%m%d')
produto_safe = PRODUTO_PAM.replace(' ', '_').replace('(', '').replace(')', '')
filename = f'dataset_mapbiomas_{NIVEL_AGREGACAO}_{produto_safe}_{ANOS[0]}-{ANOS[-1]}_{timestamp}.csv'
filepath = f'data/{filename}'

# Salvar
df_export.to_csv(filepath, index=False)

print(f"\n✅ DATASET EXPORTADO COM SUCESSO!")
print(f"📁 Arquivo: {filepath}")
print(f"📏 Tamanho: {os.path.getsize(filepath) / 1024 / 1024:.1f} MB")
print(f"📊 Dimensões: {len(df_export):,} × {len(df_export.columns)}")

# Mostrar amostra
print(f"\n🔍 Amostra dos dados:")
df_export.head(10)



✅ DATASET EXPORTADO COM SUCESSO!
📁 Arquivo: data/dataset_mapbiomas_microrregiao_Cana-de-açúcar_2003-2021_20250912.csv
📏 Tamanho: 1.6 MB
📊 Dimensões: 10,602 × 20

🔍 Amostra dos dados:


,ano,id_microrregiao,sigla_uf,primeiro_ano_tratamento,tratado,pos_tratamento,pam_area_plantada,pam_area_colhida,pam_quantidade_produzida,pam_valor_producao,pam_produtividade,populacao_total,pib_total,pib_per_capita,pib_agropecuario,area_cafe,area_cana,area_citricos,area_outras_lavouras_temporarias,area_soja
0,2003,11001,RO,2007,1,0,82,82,5695.0,226.0,69.451220,448419,3173358000,7076.769718,170001000,0.0,0.0,0.0,2.514962,0.009672
1,2004,11001,RO,2007,1,0,99,99,6817.0,291.0,68.858586,497775,3885396000,7805.526593,175586000,0.0,0.0,0.0,4.568636,0.000000
2,2005,11001,RO,2007,1,0,92,92,6111.0,271.0,66.423913,485002,4385122000,9041.451375,195425000,0.0,0.0,0.0,7.502846,0.791378
3,2006,11001,RO,2007,1,0,117,117,7645.0,440.0,65.341880,497936,4723308000,9485.773272,192598000,0.0,0.0,0.0,10.881466,0.806394
4,2007,11001,RO,2007,1,1,75,75,4657.0,318.0,62.093333,474532,5169521000,10893.935499,230175000,0.0,0.0,0.0,12.199082,0.086411
5,2008,11001,RO,2007,1,1,78,78,4576.0,327.0,58.666667,486884,6118513000,12566.675019,340318000,0.0,0.0,0.0,13.045108,0.036946
6,2009,11001,RO,2007,1,1,104,104,5827.0,233.0,56.028846,493445,7786946000,15780.778000,367499000,0.0,0.0,0.0,18.187689,0.000000
7,2010,11001,RO,2007,1,1,73,73,4088.0,163.0,56.000000,540320,10298189000,19059.425896,426759000,0.0,0.0,0.0,20.663352,0.183904
8,2011,11001,RO,2007,1,1,47,47,2587.0,760.0,55.042553,550089,12509934000,22741.654532,508095000,0.0,0.0,0.0,22.798335,2.427921
9,2012,11001,RO,2007,1,1,0,0,0.0,0.0,0.000000,559543,13425222000,23993.190872,633448000,0.0,0.0,0.0,22.288220,3.582765


In [ ]:
# Salvar metadados
metadata = {
    'nivel_agregacao': NIVEL_AGREGACAO,
    'produto_pam': PRODUTO_PAM,
    'anos': f"{ANOS[0]}-{ANOS[-1]}",
    'n_observacoes': len(df_export),
    'n_unidades': df_export[id_col].nunique(),
    'n_tratadas': df_export[df_export['tratado'] == 1][id_col].nunique(),
    'n_controle': df_export[df_export['tratado'] == 0][id_col].nunique(),
    'categorias_mapbiomas': list(CATEGORIAS_MAPBIOMAS.values()),
    'data_extracao': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

# Salvar metadados em JSON
import json
metadata_file = filepath.replace('.csv', '_metadata.json')
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"\n📝 Metadados salvos: {metadata_file}")

# Salvar também o mapeamento geográfico usado
if NIVEL_AGREGACAO == 'microrregiao':
    mapping_file = 'output/mapeamento_municipio_microrregiao.csv'
    df_municipios.to_csv(mapping_file, index=False)
    print(f"🗺️ Mapeamento salvo: {mapping_file}")
elif NIVEL_AGREGACAO == 'amc':
    mapping_file = 'output/mapeamento_municipio_amc.csv'
    df_base[df_base['id_amc'].notna()].to_csv(mapping_file, index=False)
    print(f"🗺️ Mapeamento salvo: {mapping_file}")


## 9. Dicionário de Variáveis

Criar documentação detalhada de todas as variáveis do dataset:


In [16]:
# Criar dicionário de variáveis do dataset
descricao_variaveis = pd.DataFrame([
    # Variáveis de identificação
    {'variavel': 'ano', 
     'descricao': 'Ano da observação', 
     'tipo': 'int', 
     'unidade': 'ano', 
     'fonte': 'Base dos Dados'},
    
    {'variavel': id_col, 
     'descricao': f'Código do(a) {NIVEL_AGREGACAO}', 
     'tipo': 'string', 
     'unidade': 'código IBGE', 
     'fonte': 'IBGE/Base dos Dados'},
    
    {'variavel': 'sigla_uf', 
     'descricao': 'Sigla da Unidade Federativa (estado)', 
     'tipo': 'string', 
     'unidade': 'sigla', 
     'fonte': 'IBGE/Base dos Dados'},
    
    # Variáveis de tratamento
    {'variavel': 'primeiro_ano_tratamento', 
     'descricao': 'Ano de instalação da primeira estação meteorológica automática (0 = sem estação)', 
     'tipo': 'int', 
     'unidade': 'ano', 
     'fonte': 'INMET/Base dos Dados'},
    
    {'variavel': 'tratado', 
     'descricao': 'Indicador se a unidade possui estação meteorológica (1 = sim, 0 = não)', 
     'tipo': 'int', 
     'unidade': 'binário', 
     'fonte': 'INMET/Base dos Dados'},
    
    {'variavel': 'pos_tratamento', 
     'descricao': 'Indicador se a observação é após instalação da estação (1 = sim, 0 = não)', 
     'tipo': 'int', 
     'unidade': 'binário', 
     'fonte': 'INMET/Base dos Dados'},
    
    # Variáveis PAM
    {'variavel': 'pam_area_plantada', 
     'descricao': f'Área plantada de {PRODUTO_PAM} segundo PAM', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'IBGE PAM/Base dos Dados'},
    
    {'variavel': 'pam_area_colhida', 
     'descricao': f'Área colhida de {PRODUTO_PAM} segundo PAM', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'IBGE PAM/Base dos Dados'},
    
    {'variavel': 'pam_quantidade_produzida', 
     'descricao': f'Quantidade produzida de {PRODUTO_PAM} segundo PAM', 
     'tipo': 'float', 
     'unidade': 'toneladas', 
     'fonte': 'IBGE PAM/Base dos Dados'},
    
    {'variavel': 'pam_produtividade', 
     'descricao': f'Produtividade média de {PRODUTO_PAM} (quantidade/área colhida)', 
     'tipo': 'float', 
     'unidade': 'kg/hectare', 
     'fonte': 'IBGE PAM/Base dos Dados'},
    
    {'variavel': 'pam_valor_producao', 
     'descricao': f'Valor da produção de {PRODUTO_PAM} segundo PAM', 
     'tipo': 'float', 
     'unidade': 'mil reais', 
     'fonte': 'IBGE PAM/Base dos Dados'},
    
    # Variáveis socioeconômicas
    {'variavel': 'populacao_total', 
     'descricao': f'População total do(a) {NIVEL_AGREGACAO}', 
     'tipo': 'float', 
     'unidade': 'habitantes', 
     'fonte': 'IBGE População/Base dos Dados'},
    
    {'variavel': 'pib_total', 
     'descricao': f'PIB total do(a) {NIVEL_AGREGACAO}', 
     'tipo': 'float', 
     'unidade': 'reais', 
     'fonte': 'IBGE PIB Municipal/Base dos Dados'},
    
    {'variavel': 'pib_per_capita', 
     'descricao': f'PIB per capita do(a) {NIVEL_AGREGACAO} (PIB/população)', 
     'tipo': 'float', 
     'unidade': 'reais/habitante', 
     'fonte': 'IBGE PIB Municipal/Base dos Dados'},
    
    {'variavel': 'pib_agropecuario', 
     'descricao': f'Valor adicionado bruto da agropecuária do(a) {NIVEL_AGREGACAO}', 
     'tipo': 'float', 
     'unidade': 'reais', 
     'fonte': 'IBGE PIB Municipal/Base dos Dados'},
    
    # Variáveis MapBiomas
    {'variavel': 'area_agricultura', 
     'descricao': 'Área total dedicada à agricultura (inclui todas as culturas)', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_lavoura_temporaria', 
     'descricao': 'Área dedicada a lavouras temporárias', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_soja', 
     'descricao': 'Área dedicada ao cultivo de soja', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_cana', 
     'descricao': 'Área dedicada ao cultivo de cana-de-açúcar', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_outras_lavouras_temporarias', 
     'descricao': 'Área dedicada a outras lavouras temporárias (exceto soja e cana)', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_cafe', 
     'descricao': 'Área dedicada ao cultivo de café', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_citricos', 
     'descricao': 'Área dedicada ao cultivo de cítricos (laranja, limão, tangerina)', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'}
])

# Salvar dicionário de variáveis
dict_file = filepath.replace('.csv', '_dicionario_variaveis.csv')
descricao_variaveis.to_csv(dict_file, index=False, encoding='utf-8-sig')

print(f"\n📚 DICIONÁRIO DE VARIÁVEIS EXPORTADO!")
print(f"📁 Arquivo: {dict_file}")
print(f"📊 Total de variáveis descritas: {len(descricao_variaveis)}")

# Mostrar preview
print(f"\n🔍 Preview do dicionário:")
descricao_variaveis.head(10)



📚 DICIONÁRIO DE VARIÁVEIS EXPORTADO!
📁 Arquivo: data/dataset_mapbiomas_microrregiao_Cana-de-açúcar_2003-2021_20250912_dicionario_variaveis.csv
📊 Total de variáveis descritas: 22

🔍 Preview do dicionário:


,variavel,descricao,tipo,unidade,fonte
0,ano,Ano da observação,int,ano,Base dos Dados
1,id_microrregiao,Código do(a) microrregiao,string,código IBGE,IBGE/Base dos Dados
2,sigla_uf,Sigla da Unidade Federativa (estado),string,sigla,IBGE/Base dos Dados
3,primeiro_ano_tratamento,Ano de instalação da primeira estação meteorol...,int,ano,INMET/Base dos Dados
4,tratado,Indicador se a unidade possui estação meteorol...,int,binário,INMET/Base dos Dados
5,pos_tratamento,Indicador se a observação é após instalação da...,int,binário,INMET/Base dos Dados
6,pam_area_plantada,Área plantada de Cana-de-açúcar segundo PAM,float,hectares,IBGE PAM/Base dos Dados
7,pam_area_colhida,Área colhida de Cana-de-açúcar segundo PAM,float,hectares,IBGE PAM/Base dos Dados
8,pam_quantidade_produzida,Quantidade produzida de Cana-de-açúcar segundo...,float,toneladas,IBGE PAM/Base dos Dados
9,pam_produtividade,Produtividade média de Cana-de-açúcar (quantid...,float,kg/hectare,IBGE PAM/Base dos Dados


In [ ]:
# Criar versão em Markdown do dicionário
markdown_content = f"""# Dicionário de Variáveis - Dataset MapBiomas

**Dataset**: {filename}  
**Período**: {ANOS[0]}-{ANOS[-1]}  
**Nível de agregação**: {NIVEL_AGREGACAO.title()}  
**Produto PAM**: {PRODUTO_PAM}  
**Data de geração**: {datetime.now().strftime('%d/%m/%Y')}

## Descrição das Variáveis

### 1. Variáveis de Identificação

- **ano**: Ano da observação (2003-2021)
- **{id_col}**: Código IBGE do(a) {NIVEL_AGREGACAO}
- **sigla_uf**: Sigla da Unidade Federativa (estado)

### 2. Variáveis de Tratamento (Estações Meteorológicas)

- **primeiro_ano_tratamento**: Ano de instalação da primeira estação meteorológica automática na unidade geográfica. Valor 0 indica ausência de estação.
- **tratado**: Variável binária indicando se a unidade possui estação meteorológica (1 = sim, 0 = não)
- **pos_tratamento**: Variável binária indicando se a observação ocorre após a instalação da estação (1 = sim, 0 = não)

### 3. Variáveis de Produção Agrícola (PAM - Produção Agrícola Municipal)

- **pam_area_plantada**: Área plantada de {PRODUTO_PAM} em hectares (autodeclarada)
- **pam_area_colhida**: Área colhida de {PRODUTO_PAM} em hectares (autodeclarada)
- **pam_quantidade_produzida**: Quantidade produzida de {PRODUTO_PAM} em toneladas
- **pam_produtividade**: Produtividade média em kg/hectare (quantidade/área colhida)
- **pam_valor_producao**: Valor da produção em mil reais

### 4. Variáveis Socioeconômicas

- **populacao_total**: População total do(a) {NIVEL_AGREGACAO} (habitantes)
- **pib_total**: PIB total em reais correntes
- **pib_per_capita**: PIB per capita em reais/habitante
- **pib_agropecuario**: Valor adicionado bruto da agropecuária em reais

### 5. Variáveis de Uso da Terra (MapBiomas - Dados de Satélite)

- **area_agricultura**: Área total dedicada à agricultura, incluindo todas as culturas (hectares)
- **area_lavoura_temporaria**: Área dedicada a lavouras temporárias (hectares)
- **area_soja**: Área específica de cultivo de soja (hectares)
- **area_cana**: Área específica de cultivo de cana-de-açúcar (hectares)
- **area_outras_lavouras_temporarias**: Área de outras lavouras temporárias excluindo soja e cana (hectares)
- **area_cafe**: Área específica de cultivo de café (hectares)
- **area_citricos**: Área específica de cultivo de cítricos (hectares)

## Notas Metodológicas

1. **MapBiomas vs PAM**: Os dados do MapBiomas são baseados em imagens de satélite e tendem a ser mais precisos que os dados autodeclarados da PAM.

2. **Agregação**: Quando o nível de agregação é superior ao município, os valores são somados (exceto produtividade, que é recalculada).

3. **Valores faltantes**: 
   - Zeros indicam ausência real do fenômeno (ex: sem produção)
   - NAs foram removidos apenas para população e PIB

4. **Tratamento**: O tratamento refere-se à presença de estações meteorológicas automáticas do INMET, que fornecem dados climáticos mais precisos e frequentes.

## Fontes dos Dados

- **IBGE**: População, PIB Municipal, Produção Agrícola Municipal (PAM)
- **INMET**: Estações meteorológicas
- **MapBiomas**: Cobertura e uso da terra
- **Base dos Dados**: Plataforma de acesso integrado aos dados

---
*Documento gerado automaticamente pelo notebook de extração de dados*
"""

# Salvar arquivo Markdown
md_file = filepath.replace('.csv', '_dicionario_variaveis.md')
with open(md_file, 'w', encoding='utf-8') as f:
    f.write(markdown_content)

print(f"\n📝 DICIONÁRIO EM MARKDOWN EXPORTADO!")
print(f"📁 Arquivo: {md_file}")
print(f"\n✅ Arquivos de documentação criados com sucesso!")
print("   - CSV com tabela de variáveis")
print("   - Markdown com descrição detalhada")
print("   - JSON com metadados do dataset")
